In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import os
import glob
import pickle
import pandas as pd
import csv
from pathlib import Path
from IPython.display import HTML
from os import listdir, mkdir

from IPython import display

import tensorflow as tf
from keras.models import load_model
from keras import layers 
from keras.layers import Concatenate,Input, Dense, Activation, ZeroPadding2D, \
BatchNormalization, Flatten, Conv2D 
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, \
GlobalAveragePooling2D 
from keras.models import Model, Sequential 
from keras.preprocessing import image 
from keras.utils import layer_utils 
from keras.utils.data_utils import get_file 
from keras.applications.imagenet_utils import preprocess_input 
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [2]:
path0 = "D:/cancer de pele/" 

In [3]:
dataf = pd.read_csv(path0+'cancer-targets.csv')
dataf.head(15)

,Unnamed: 0,image_id,akiec,bcc,mel,nv,bkl,df,vasc
0,0,ISIC_0027419,0,0,0,0,1,0,0
1,1,ISIC_0025030,0,0,0,0,1,0,0
2,2,ISIC_0026769,0,0,0,0,1,0,0
3,3,ISIC_0025661,0,0,0,0,1,0,0
4,4,ISIC_0031633,0,0,0,0,1,0,0
5,5,ISIC_0027850,0,0,0,0,1,0,0
6,6,ISIC_0029176,0,0,0,0,1,0,0
7,7,ISIC_0029068,0,0,0,0,1,0,0
8,8,ISIC_0025837,0,0,0,0,1,0,0
9,9,ISIC_0025209,0,0,0,0,1,0,0


In [ ]:
id_img_unique = pd.unique(dataf['image_id'])
print("Número de imagens únicas: ", len(id_img_unique))

In [ ]:
os.chdir(path0+"data_pickle/")
files=glob.glob("*.pckl")
print("Número de arquivos lidos: ", len(files))

In [ ]:
fig = plt.figure(figsize=(10,12))
X   = np.zeros((len(files),250,250,3))

for j in range(0,len(files)):
   f = open(files[j], 'rb')
   X[j] = pickle.load(f)

In [ ]:
imagens = rd.choice(range(len(files)), size = 134, replace = False)

In [ ]:
#Checar gatos 
fig = plt.figure(figsize=(9,12))
for j in range(0,16):
    fig.add_subplot(4,4,j+1).imshow(X[j])
    plt.title("file: " + files[j][:-4])   

In [ ]:
def KerasModel(input_shape):  
    NumClass = 7
    X_input  = Input(input_shape)
     
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(40, (3, 3), strides = (1, 1), name = 'conv0')(X)    
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    
    ##
    X = Conv2D(24, (3, 3), strides = (1, 1), name = 'conv1')(X)    
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool1')(X)  
    ##
    
    ##
    X = Conv2D(16, (3, 3), strides = (1, 1), name = 'conv2')(X)    
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool2')(X)  
    ##    

    ##   New layer
    X = Conv2D(8, (3, 3), strides = (1, 1), name = 'conv3')(X)    
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool3')(X)  
    ##    


    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(NumClass, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='KerasModel')
    
    return model 

In [ ]:
# Construct the model
ModelMancha = KerasModel(X[0].shape) # input dimention
# Run the model
ModelMancha.compile(loss='binary_crossentropy',optimizer='Adam', metrics=['accuracy'])

In [ ]:
history = ModelMancha.fit( x=X, y=Y, batch_size=2**6, epochs=20) ## batch = 2**6
plt.plot(history.history['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()